In [1]:
from bs4 import BeautifulSoup
import lxml
import html5lib
import re
import json
import uuid
import glob
import pickle
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.lancaster import LancasterStemmer
from nltk.cluster import KMeansClusterer, euclidean_distance
from gensim import corpora, models, utils
from numpy import array
from gensim.parsing.preprocessing import STOPWORDS
from gensim.utils import simple_preprocess
from pprint import pprint
from gensim import corpora
import re
from os.path import basename
import string
import logging
import pandas as pd
import numpy as np
import pyLDAvis
from empath import Empath
lexicon = Empath()

In [2]:
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

DEBUG:root:test


In [3]:
pd.set_option("display.max_colwidth", -1)
pd.set_option("display.max_columns", 100)

In [4]:
RedditComplete = pd.read_pickle('CompleteParentingAddedTokenized.pkl')

In [5]:
topiclists = []

In [48]:
#These topic names could be filled in wiht the topics that arise from the LDA model
Topics_Name = ["Topic 1", "Topic 2", "Topic 3", "Topic 4", "Topic 5", "Topic 6","Topic 7",\
           "Topic 8","Topic 9","Topic 10","Topic 11","Topic 12","Topic 13","Topic 14","Topic 15",\
           "Topic 16","Topic 17","Topic 18","Topic 19","Topic 20","Topic 21","Topic 22","Topic 23",\
           "Topic 24","Topic 25","Topic 26","Topic 27","Topic 28","Topic 29","Topic 30","Topic 31",\
           "Topic 32","Topic 33","Topic 34","Topic 35","Topic 36","Topic 37","Topic 38","Topic 39","Topic 40",\
           "Topic 41","Topic 42","Topic 43","Topic 44","Topic 45","Topic 46","Topic 47","Topic 48","Topic 49","Topic 50"]

In [7]:
def normalize_topic(intersection, TextList):
    if len(TextList)==0:
        return 0
    else:
        return float(len(intersection)/len(TextList+intersection))

In [8]:
def empathize(text, category):
    return(lexicon.analyze(text, categories=[category], normalize=True))

In [9]:
def find_intersection(TextList,LDAList):
    intersection = list(set(TextList) & set(LDAList))
    return normalize_topic(intersection,TextList)

In [10]:
def Tokinization(document):
    document = "".join(document)
    tokenizer = RegexpTokenizer(r'\w+')
    intermediate = tokenizer.tokenize(document)
    #stop words
    stop = stopwords.words('english')
    intermediate = [i for i in intermediate if i not in stop]
    #stemming
    lanste = LancasterStemmer()
    intermediate = [lanste.stem(i) for i in intermediate]
    return intermediate

In [11]:
def Preprocess(document):
    #document = document.encode('ascii','ignore')
    #remove http
    intermediate = document
    #join
    intermediate = "".join(document)
    intermediate = re.sub(r'htt\S+', '', intermediate)
    intermediate = re.sub(r'://t.co\S*', '', intermediate)
    # remove rts
    intermediate = re.sub(r'\brt\b','', intermediate)
    # remove digits
    intermediate = re.sub(r'\d+','', intermediate)
    # remove names after @
    # (screen names)
    intermediate = re.sub("@\\w+ *", '', intermediate)
    intermediate = re.sub(r'\b@\b','', intermediate)
    intermediate = unescape(intermediate)
    exclude = set(string.punctuation)
    intermediate = "".join(ch for ch in intermediate if ch is not exclude)
    
    intermediate = Tokinization(intermediate)
    return intermediate

In [12]:
def clean_LDA_topics(LDATopics):
    addch = "'"
    topics = []
    for i in range(0,len(LDATopics)):
        current_topic  = LDATopics[i][1]
        print(current_topic)
        newwords = []
        split_current_topic = []
        split_current_topic = current_topic.split("+")
        for j in range(0, len(split_current_topic)):
            newword = split_current_topic[j]
            print(split_current_topic)
            if j==0:
                newword = newword[7:-2]
            else:
                newword = newword[8:-2]
                newword = newword[:0] + newword[0:]
            newwords.append(newword)
        topics.append(newwords)
    return topics

In [13]:
def removeNone(intermediate):
    #remove None Values
    intermediate = ['' if v is None else v for v in intermediate]
    return intermediate

In [14]:
def unescape(s):
    s = s.replace("&gt;", "greater than")
    s = s.replace("&lt;3", "love")
    s = s.replace("&lt;", "less than")
    s = s.replace("&amp;", "and")
    s = s.replace("via", '')
    s = s.replace('\\n', '')
    s = s.replace('\\t', '')
    s = s.replace('\\r', '')
    return s

In [15]:
#lower strings in all comment body
RedditComplete['body'] = RedditComplete['body'].str.lower()
#create lists of the lower string body
bodylist = RedditComplete['body'].tolist()
#remove None
NoneRemovedbodylist = removeNone(bodylist)
#create the new body
RedditComplete['body']=NoneRemovedbodylist
#create threads from the comments
#through grouping by link_id
CompleteThread = RedditComplete.groupby('link_id')['body'].apply(list)

In [16]:
#running for the posts before breaking to threads
#these will be used to calculate Jaccard similarity
#betwee the comment and the LDA Linguistic Group (LG)
processed_posts = []
#creating threads for each of the users
for post in NoneRemovedbodylist:
    #Preprocessing each of the posts
    processed_posts.append(Preprocess(post))

In [17]:
#running for the rest of the data
processed_threads = []
#creating threads for each of the users
for thread in CompleteThread:
    #Preprocessing each of the threads
    processed_threads.append(Preprocess(thread))

In [18]:
RedditComplete['body_tokenized'] = processed_posts

In [19]:
texts = processed_threads

In [24]:
# Get token frequencies
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token]+=1

In [25]:
tokenized = [[token for token in text if frequency[token]>1]for text in texts]

In [26]:
wordfreq = {}
for text in texts:
    for token in text:
        wordfreq[token]=frequency[token]

In [28]:
higherfreq = {k:v for k, v in wordfreq.items() if v>10000}  

In [29]:
import operator
x = higherfreq2
sorted_x = sorted(x.items(), key=operator.itemgetter(1))
sorted_x

[('key', 10015),
 ('lady', 10038),
 ('suff', 10044),
 ('brand', 10045),
 ('known', 10078),
 ('ridic', 10105),
 ('loss', 10114),
 ('sud', 10122),
 ('art', 10151),
 ('fed', 10153),
 ('account', 10181),
 ('random', 10186),
 ('uncomfort', 10199),
 ('wrap', 10295),
 ('increas', 10327),
 ('star', 10339),
 ('diet', 10346),
 ('feet', 10348),
 ('disagr', 10406),
 ('conceiv', 10411),
 ('company', 10436),
 ('gonn', 10491),
 ('submit', 10498),
 ('styl', 10506),
 ('dry', 10506),
 ('pag', 10513),
 ('charg', 10534),
 ('paid', 10538),
 ('b', 10539),
 ('jok', 10553),
 ('comput', 10561),
 ('ground', 10570),
 ('laundry', 10584),
 ('custody', 10596),
 ('am', 10603),
 ('grab', 10630),
 ('eg', 10640),
 ('tear', 10650),
 ('dec', 10658),
 ('addit', 10663),
 ('log', 10664),
 ('int', 10671),
 ('confid', 10678),
 ('dea', 10695),
 ('bunch', 10729),
 ('ahead', 10738),
 ('expos', 10756),
 ('red', 10759),
 ('text', 10775),
 ('exhaust', 10780),
 ('bil', 10781),
 ('chair', 10799),
 ('pre', 10809),
 ('grandm', 10847),


In [30]:
dictionary = corpora.Dictionary(texts)
dictionary.save('CompleteFinal.dict')
print(dictionary)

INFO:gensim.corpora.dictionary:adding document #0 to Dictionary(0 unique tokens: [])
INFO:gensim.corpora.dictionary:adding document #10000 to Dictionary(37013 unique tokens: ['learn', 'langu', 'ear', 'lik', 'sign']...)
INFO:gensim.corpora.dictionary:adding document #20000 to Dictionary(54351 unique tokens: ['learn', 'langu', 'ear', 'lik', 'sign']...)
INFO:gensim.corpora.dictionary:adding document #30000 to Dictionary(71892 unique tokens: ['learn', 'langu', 'ear', 'lik', 'sign']...)
INFO:gensim.corpora.dictionary:adding document #40000 to Dictionary(87048 unique tokens: ['learn', 'langu', 'ear', 'lik', 'sign']...)
INFO:gensim.corpora.dictionary:adding document #50000 to Dictionary(101179 unique tokens: ['learn', 'langu', 'ear', 'lik', 'sign']...)
INFO:gensim.corpora.dictionary:adding document #60000 to Dictionary(113945 unique tokens: ['learn', 'langu', 'ear', 'lik', 'sign']...)
INFO:gensim.corpora.dictionary:adding document #70000 to Dictionary(125423 unique tokens: ['learn', 'langu', 

Dictionary(169995 unique tokens: ['learn', 'langu', 'ear', 'lik', 'sign']...)


In [31]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('CompleteCorpusFinal.mm', corpus)

INFO:gensim.corpora.mmcorpus:storing corpus in Matrix Market format to CompleteCorpusFinal.mm
INFO:gensim.matutils:saving sparse matrix to CompleteCorpusFinal.mm
INFO:gensim.matutils:PROGRESS: saving document #0
INFO:gensim.matutils:PROGRESS: saving document #1000
INFO:gensim.matutils:PROGRESS: saving document #2000
INFO:gensim.matutils:PROGRESS: saving document #3000
INFO:gensim.matutils:PROGRESS: saving document #4000
INFO:gensim.matutils:PROGRESS: saving document #5000
INFO:gensim.matutils:PROGRESS: saving document #6000
INFO:gensim.matutils:PROGRESS: saving document #7000
INFO:gensim.matutils:PROGRESS: saving document #8000
INFO:gensim.matutils:PROGRESS: saving document #9000
INFO:gensim.matutils:PROGRESS: saving document #10000
INFO:gensim.matutils:PROGRESS: saving document #11000
INFO:gensim.matutils:PROGRESS: saving document #12000
INFO:gensim.matutils:PROGRESS: saving document #13000
INFO:gensim.matutils:PROGRESS: saving document #14000
INFO:gensim.matutils:PROGRESS: saving doc

In [32]:
#Create LDA model
CompleteLDA = models.ldamodel.LdaModel
#Set the hyper parameters
ldamodelComplete = CompleteLDA(corpus, num_topics=50, id2word=dictionary, passes=20)

INFO:gensim.models.ldamodel:using symmetric alpha at 0.02
INFO:gensim.models.ldamodel:using symmetric eta at 5.8825259566457836e-06
INFO:gensim.models.ldamodel:using serial LDA version on this node
INFO:gensim.models.ldamodel:running online LDA training, 50 topics, 20 passes over the supplied corpus of 114044 documents, updating model once every 2000 documents, evaluating perplexity every 20000 documents, iterating 50x with a convergence threshold of 0.001000
INFO:gensim.models.ldamodel:PROGRESS: pass 0, at document #2000/114044
DEBUG:gensim.models.ldamodel:performing inference on a chunk of 2000 documents
DEBUG:gensim.models.ldamodel:506/2000 documents converged within 50 iterations
DEBUG:gensim.models.ldamodel:updating topics
INFO:gensim.models.ldamodel:merging changes from 2000 documents into a model of 114044 documents
INFO:gensim.models.ldamodel:topic #27 (0.020): 0.013*"get" + 0.011*"kid" + 0.010*"tim" + 0.009*"lik" + 0.009*"us" + 0.008*"real" + 0.008*"tak" + 0.007*"know" + 0.007

In [34]:
ldamodelComplete.save('CompleteRedditLDAModelFinal.model')

INFO:gensim.utils:saving LdaState object under CompleteRedditLDAModelFinal.model.state, separately None
INFO:gensim.utils:saved CompleteRedditLDAModelFinal.model.state
INFO:gensim.utils:saving LdaModel object under CompleteRedditLDAModelFinal.model, separately ['expElogbeta', 'sstats']
INFO:gensim.utils:storing np array 'expElogbeta' to CompleteRedditLDAModelFinal.model.expElogbeta.npy
INFO:gensim.utils:not storing attribute state
INFO:gensim.utils:not storing attribute dispatcher
INFO:gensim.utils:not storing attribute id2word
INFO:gensim.utils:saved CompleteRedditLDAModelFinal.model


In [35]:
LDA_topics= ldamodelComplete.print_topics(num_topics=50, num_words=30)

INFO:gensim.models.ldamodel:topic #0 (0.020): 0.069*"seat" + 0.044*"car" + 0.026*"fac" + 0.019*"us" + 0.017*"rear" + 0.012*"saf" + 0.011*"instal" + 0.011*"cars" + 0.011*"belt" + 0.009*"forward" + 0.009*"driv" + 0.008*"fit" + 0.008*"child" + 0.007*"boost" + 0.007*"weight" + 0.007*"check" + 0.007*"get" + 0.007*"lik" + 0.007*"lbs" + 0.007*"back" + 0.006*"front" + 0.006*"on" + 0.006*"sit" + 0.006*"pound" + 0.006*"crash" + 0.005*"nee" + 0.005*"tal" + 0.005*"leg" + 0.005*"year" + 0.005*"mak"
INFO:gensim.models.ldamodel:topic #1 (0.020): 0.040*"us" + 0.032*"on" + 0.022*"carry" + 0.019*"stor" + 0.019*"stroller" + 0.016*"get" + 0.014*"lik" + 0.014*"buy" + 0.012*"lov" + 0.012*"easy" + 0.011*"also" + 0.011*"real" + 0.010*"look" + 0.010*"baby" + 0.009*"good" + 0.009*"gre" + 0.009*"shop" + 0.009*"got" + 0.008*"nee" + 0.008*"wrap" + 0.007*"lot" + 0.007*"bought" + 0.007*"much" + 0.007*"find" + 0.007*"work" + 0.006*"cheap" + 0.006*"go" + 0.006*"back" + 0.006*"cart" + 0.006*"fit"
INFO:gensim.models.lda

In [36]:
All_topics = clean_LDA_topics(LDA_topics)

0.069*"seat" + 0.044*"car" + 0.026*"fac" + 0.019*"us" + 0.017*"rear" + 0.012*"saf" + 0.011*"instal" + 0.011*"cars" + 0.011*"belt" + 0.009*"forward" + 0.009*"driv" + 0.008*"fit" + 0.008*"child" + 0.007*"boost" + 0.007*"weight" + 0.007*"check" + 0.007*"get" + 0.007*"lik" + 0.007*"lbs" + 0.007*"back" + 0.006*"front" + 0.006*"on" + 0.006*"sit" + 0.006*"pound" + 0.006*"crash" + 0.005*"nee" + 0.005*"tal" + 0.005*"leg" + 0.005*"year" + 0.005*"mak"
['0.069*"seat" ', ' 0.044*"car" ', ' 0.026*"fac" ', ' 0.019*"us" ', ' 0.017*"rear" ', ' 0.012*"saf" ', ' 0.011*"instal" ', ' 0.011*"cars" ', ' 0.011*"belt" ', ' 0.009*"forward" ', ' 0.009*"driv" ', ' 0.008*"fit" ', ' 0.008*"child" ', ' 0.007*"boost" ', ' 0.007*"weight" ', ' 0.007*"check" ', ' 0.007*"get" ', ' 0.007*"lik" ', ' 0.007*"lbs" ', ' 0.007*"back" ', ' 0.006*"front" ', ' 0.006*"on" ', ' 0.006*"sit" ', ' 0.006*"pound" ', ' 0.006*"crash" ', ' 0.005*"nee" ', ' 0.005*"tal" ', ' 0.005*"leg" ', ' 0.005*"year" ', ' 0.005*"mak"']
['0.069*"seat" ', '

In [37]:
topiclist = All_topics

In [49]:
#Calculate the Jaccard score for each of the comments and append them into a list
currentList = []
mega_topic_list= [[] for _ in range(len(topiclist)) ]
for i in range(0, len(mega_topic_list)):
    currentList = mega_topic_list[i]
    currentTopic = topiclist[i]
    for j in range (0, len(processed_posts)):
        intersected = find_intersection(processed_posts[j], currentTopic)
        currentList.append(intersected)

In [50]:
#This is how the list looks like
#Each of the numbers represents the 
#Jaccard similarity of the comment and
#the topic
currentList

[0.0,
 0.0,
 0.07692307692307693,
 0.0,
 0.0,
 0.09375,
 0.0,
 0.0,
 0.0,
 0.0,
 0.07692307692307693,
 0.0,
 0.1,
 0.0,
 0.07142857142857142,
 0.015873015873015872,
 0.0,
 0.0,
 0.0,
 0.034482758620689655,
 0.05405405405405406,
 0.0410958904109589,
 0.04878048780487805,
 0.030303030303030304,
 0.0,
 0.09333333333333334,
 0.16666666666666666,
 0.0,
 0.25,
 0.1,
 0.5,
 0.047619047619047616,
 0.16666666666666666,
 0,
 0.0,
 0.13333333333333333,
 0.0,
 0,
 0.0,
 0.0,
 0,
 0,
 0.07272727272727272,
 0.0,
 0.1,
 0,
 0,
 0.06896551724137931,
 0.0,
 0.0,
 0.075,
 0,
 0.037037037037037035,
 0.0,
 0.0,
 0.038461538461538464,
 0.0,
 0.0,
 0.0547945205479452,
 0.0625,
 0.0,
 0.0,
 0.3333333333333333,
 0.0625,
 0.0,
 0,
 0.019230769230769232,
 0.058823529411764705,
 0.0,
 0.05128205128205128,
 0.08,
 0.0,
 0.14285714285714285,
 0,
 0.058823529411764705,
 0.09090909090909091,
 0.1,
 0.06666666666666667,
 0.0851063829787234,
 0.053763440860215055,
 0.07692307692307693,
 0.07407407407407407,
 0.0340909

In [83]:
#Adding the scores equivalent to each column
for i in range (0,len(Topics_Name)):
    RedditComplete[Topics_Name[i]]= mega_topic_list[i]

In [84]:
RedditComplete.head()

,archived,author,author_flair_css_class,author_flair_text,body,controversiality,created_utc,distinguished,downs,edited,gilded,link_id,name,parent_id,removal_reason,score,score_hidden,subreddit,ups,body_tokenized,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6,Topic 7,Topic 8,Topic 9,Topic 10,Topic 11,Topic 12,Topic 13,Topic 14,Topic 15,Topic 16,Topic 17,Topic 18,Topic 19,Topic 20,Topic 21,Topic 22,Topic 23,Topic 24,Topic 25,Topic 26,Topic 27,Topic 28,Topic 29,Topic 30,Topic 31,Topic 32,Topic 33,Topic 34,Topic 35,Topic 36,Topic 37,Topic 38,Topic 39,Topic 40,Topic 41,Topic 42,Topic 43,Topic 44,Topic 45,Topic 46,Topic 47,Topic 48,Topic 49,Topic 50
0,True,kickstand,None,None,now you tell me. :-(,0,2008-04-09 15:16:05,None,0.0,NaT,0,t3_6d8mw,t1_c03o5z1,t3_6d8mw,NaN,1,False,Parenting,1.0,[tel],0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
1,True,kickstand,None,None,"at our house, the tooth fairy leaves one shiny $1 coin with a dead white president's face on it.\n\nhttp://www.usmint.gov/mint_programs/$1coin/index.cfm?flash=yes",0,2008-04-09 15:17:46,None,0.0,NaT,0,t3_6d8mh,t1_c03o5zp,t3_6d8mh,NaN,1,False,Parenting,1.0,"[hous, too, fairy, leav, on, shiny, coin, dead, whit, presid, fac]",0.181818,0.090909,0.090909,0.090909,0.181818,0.181818,0.181818,0.000000,0.090909,0.000000,0.090909,0.090909,0.090909,0.090909,0.090909,0.181818,0.090909,0.0,0.000000,0.000000,0.090909,0.090909,0.090909,0.090909,0.181818,0.090909,0.090909,0.090909,0.000000,0.090909,0.090909,0.000000,0.000000,0.090909,0.000000,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
2,True,Haven,None,None,"good post. i wonder though how old his kids are. i created a personal website for my 2nd grader, with all his favorite links on it.",0,2008-08-05 20:11:57,None,0.0,NaT,0,t3_6v0tv,t1_c04xv27,t3_6v0tv,NaN,3,False,Parenting,3.0,"[good, post, wond, though, old, kid, cre, person, websit, nd, grad, favorit, link]",0.000000,0.076923,0.076923,0.153846,0.000000,0.153846,0.000000,0.153846,0.307692,0.230769,0.000000,0.000000,0.153846,0.230769,0.076923,0.076923,0.000000,0.0,0.000000,0.076923,0.153846,0.076923,0.000000,0.000000,0.076923,0.000000,0.230769,0.000000,0.076923,0.076923,0.153846,0.153846,0.076923,0.076923,0.076923,0.000000,0.153846,0.076923,0.076923,0.000000,0.076923,0.076923,0.230769,0.000000,0.153846,0.000000,0.076923,0.0,0.000000,0.076923
3,True,None,None,None,"the author makes his point succinctly in the comments section -\n\n&gt;until they can hack my proxy server and erase the squid logs, they won't be able to hide their history from me. as soon as that happens there won't be anything i can do to protect them, so we may as well teach them proper behavior in the first place.",0,2008-08-06 01:09:41,None,0.0,NaT,0,t3_6v0tv,t1_c04xyt2,t3_6v0tv,NaN,2,False,Parenting,2.0,"[auth, mak, point, succinct, com, sect, gre, thanuntil, hack, proxy, serv, eras, squid, log, abl, hid, hist, soon, hap, anyth, protect, may, wel, teach, prop, behavy, first, plac]",0.000000,0.035714,0.000000,0.035714,0.035714,0.035714,0.000000,0.000000,0.035714,0.035714,0.035714,0.000000,0.178571,0.071429,0.000000,0.000000,0.035714,0.0,0.035714,0.035714,0.035714,0.035714,0.035714,0.000000,0.035714,0.035714,0.071429,0.071429,0.071429,0.071429,0.071429,0.035714,0.000000,0.071429,0.035714,0.071429,0.035714,0.071429,0.000000,0.035714,0.000000,0.035714,0.035714,0.035714,0.035714,0.035714,0.071429,0.0,0.035714,0.000000
4,True,Sammzor,None,None,[guys can breastfeed too.](http://en.wikipedia.org/wiki/male_lactation)\n,0,2008-08-08 04:14:31,None,0.0,1970